In [ ]:
import json
import requests
import pandas as pd

def fetch_data(api_url):
    """Fetch data from API."""
    try:
        response = requests.get(api_url, timeout=10)
        response.raise_for_status()  # Raise HTTP errors
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

def save_to_csv(data, filename="exported_electricity.csv"):
    """Process and save data to CSV."""
    if 'unix_seconds' not in data or 'countries' not in data:
        print("Unexpected data format")
        return

    timestamps = pd.to_datetime(data['unix_seconds'], unit='s')

    sum_data = None
    for country in data['countries']:
        if isinstance(country, dict) and country.get('name') == 'sum':
            sum_data = country.get('data', [])
            break

    if not sum_data or len(sum_data) != len(timestamps):
        print("No valid 'sum' data found")
        return

    df = pd.DataFrame({'date': timestamps, 'sum': sum_data})
    df.set_index('date', inplace=True)
    
    df['sum'] = df['sum'].abs()
    
    # Save to CSV
    df.to_csv(filename)
    print(f"Data saved to {filename}")

    # Aggregate by month and year
    df['year-month'] = df.index.to_period('M').astype(str)  # Convert to YYYY-MM format
    monthly_summary = df.groupby('year-month')['sum'].agg(['min', 'max', 'mean'])

    # Save aggregated data to CSV
    summary_file = "monthly_summary.csv"
    monthly_summary.to_csv(summary_file)
    print(f"Monthly summary saved to {summary_file}")

# Define the API endpoint
api_url = 'https://api.energy-charts.info/cbet?country=se&start=2019-01-01&end=2024-12-31'

# Fetch and save the data
data = fetch_data(api_url)
if data:
    save_to_csv(data)

Data saved to exported_electricity.csv
Monthly summary saved to monthly_summary.csv
